In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from typing import List, Dict, Optional

# =========================================================
# 1) NATIJALAR — RAQAMLAR KOD ICHIDA (hard-coded)
# =========================================================

baseline_rows: List[Dict] = [
    {"model": "baseline_ovr_linearsvc",            "test_micro_f1": 0.999500, "test_macro_f1": 0.998800, "test_micro_precision": 0.999200, "test_micro_recall": 0.999800},
    {"model": "baseline_ovr_calibrated_linearsvc", "test_micro_f1": 0.999200, "test_macro_f1": 0.998400, "test_micro_precision": 0.998900, "test_micro_recall": 0.999500},
    {"model": "baseline_ovr_logreg",               "test_micro_f1": 0.997300, "test_macro_f1": 0.995800, "test_micro_precision": 0.996000, "test_micro_recall": 0.998600},
    {"model": "baseline_ovr_sgd_logloss",          "test_micro_f1": 0.995000, "test_macro_f1": 0.992000, "test_micro_precision": 0.993400, "test_micro_recall": 0.996600},
]

improvement_rows: List[Dict] = [
    {"model": "ovr_logreg_bal_C2",     "val_micro_f1": 0.980376, "val_macro_f1": 0.965836},
    {"model": "ovr_linearsvc_C1",      "val_micro_f1": 0.980182, "val_macro_f1": 0.964417},
    {"model": "ovr_logreg_bal_C1",     "val_micro_f1": 0.978403, "val_macro_f1": 0.944185},
    {"model": "ovr_sgd_logloss",       "val_micro_f1": 0.974033, "val_macro_f1": 0.952951},
    {"model": "ovr_sgd_hinge",         "val_micro_f1": 0.957535, "val_macro_f1": 0.936905},
    {"model": "ovr_complementnb_a05",  "val_micro_f1": 0.913360, "val_macro_f1": 0.882115},
]

tuning_rows: List[Dict] = [
    {"model": "optuna_logreg",      "val_micro_f1": 0.962941, "val_macro_f1": 0.930969, "test_micro_f1": 0.957416, "test_macro_f1": 0.916767},
    {"model": "optuna_linearsvc",   "val_micro_f1": 0.961265, "val_macro_f1": 0.908998, "test_micro_f1": 0.957016, "test_macro_f1": 0.912317},
    {"model": "optuna_sgd_logloss", "val_micro_f1": 0.950963, "val_macro_f1": 0.916381, "test_micro_f1": 0.943415, "test_macro_f1": 0.903986},
    {"model": "optuna_sgd_hinge",   "val_micro_f1": 0.948021, "val_macro_f1": 0.888889, "test_micro_f1": 0.938087, "test_macro_f1": 0.895398},
]

best_model_rows: List[Dict] = [
    {"model": "optuna_logreg_best_final", "val_micro_f1": 0.984831, "val_macro_f1": 0.973785, "test_micro_f1": 0.978199, "test_macro_f1": 0.966022},
]

# =========================================================
# 2) TABULATE + 🟩/🟥 (har jadval ichida ustun bo‘yicha max/min)
# =========================================================

def mark_max_min(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    """df nusxasini qaytaradi: ko‘rsatilgan cols ichida max 🟩, min 🟥 qo‘shadi."""
    out = df.copy()
    for c in cols:
        if c not in out.columns:
            continue
        s = pd.to_numeric(out[c], errors="coerce")
        if s.isna().all():
            continue
        vmax, vmin = s.max(), s.min()

        def fmt(v):
            if pd.isna(v):
                return "—"
            v = float(v)
            txt = f"{v:.6f}"
            if v == vmax and v == vmin:
                return txt + "🟩🟥"
            if v == vmax:
                return txt + "🟩"
            if v == vmin:
                return txt + "🟥"
            return txt

        out[c] = s.map(fmt)
    return out

def show_stage(title: str, rows: List[Dict], mark_cols: List[str], sort_by: Optional[str] = None):
    print("\n" + "=" * 90)
    print(title)
    print("=" * 90)

    df = pd.DataFrame(rows)
    if df.empty:
        print("(Natija yo‘q)")
        return

    if sort_by is not None and sort_by in df.columns:
        df = df.sort_values(sort_by, ascending=False).reset_index(drop=True)

    df_disp = mark_max_min(df, mark_cols)
    print(tabulate(df_disp, headers="keys", tablefmt="github", showindex=False))

# =========================================================
# 3) BOSQICHMA-BOSQICH CHIQARISH (tagma-tag)
# =========================================================

show_stage(
    "BASELINE — TEST metrics",
    baseline_rows,
    mark_cols=["test_micro_f1", "test_macro_f1", "test_micro_precision", "test_micro_recall"],
    sort_by="test_micro_f1"
)

show_stage(
    "IMPROVEMENT — VAL metrics",
    improvement_rows,
    mark_cols=["val_micro_f1", "val_macro_f1"],
    sort_by="val_micro_f1"
)

show_stage(
    "TUNING (NO_OVERSAMPLING) — VAL/TEST metrics",
    tuning_rows,
    mark_cols=["val_micro_f1", "val_macro_f1", "test_micro_f1", "test_macro_f1"],
    sort_by="test_micro_f1"
)

show_stage(
    "BEST MODEL (FINAL TRAIN) — VAL/TEST metrics",
    best_model_rows,
    mark_cols=["val_micro_f1", "val_macro_f1", "test_micro_f1", "test_macro_f1"],
    sort_by="test_micro_f1"
)

# =========================================================
# 4) (Qo‘shimcha) BEST-OF-STAGE — yonma-yon (HARD-CODED)
# =========================================================

best_of_stage_rows: List[Dict] = [
    {"stage": "baseline",           "model": "baseline_ovr_linearsvc",       "val_micro_f1": np.nan,   "val_macro_f1": np.nan,   "test_micro_f1": 0.999500, "test_macro_f1": 0.998800},
    {"stage": "improvement",        "model": "ovr_logreg_bal_C2",            "val_micro_f1": 0.980376, "val_macro_f1": 0.965836, "test_micro_f1": np.nan,   "test_macro_f1": np.nan},
    {"stage": "tuning",             "model": "optuna_logreg",                "val_micro_f1": 0.962941, "val_macro_f1": 0.930969, "test_micro_f1": 0.957416, "test_macro_f1": 0.916767},
    {"stage": "best_model (final)", "model": "optuna_logreg_best_final",     "val_micro_f1": 0.984831, "val_macro_f1": 0.973785, "test_micro_f1": 0.978199, "test_macro_f1": 0.966022},
]

print("\n" + "=" * 90)
print("BEST-OF-STAGE — Bosqichlar bo‘yicha “eng yaxshi”ni yonma-yon taqqoslash (hard-coded)")
print("=" * 90)

df_best_of_stage = pd.DataFrame(best_of_stage_rows)
df_best_of_stage_disp = mark_max_min(
    df_best_of_stage,
    cols=["val_micro_f1", "val_macro_f1", "test_micro_f1", "test_macro_f1"]
)

print(tabulate(df_best_of_stage_disp, headers="keys", tablefmt="github", showindex=False))


BASELINE — TEST metrics
| model                             | test_micro_f1   | test_macro_f1   | test_micro_precision   | test_micro_recall   |
|-----------------------------------|-----------------|-----------------|------------------------|---------------------|
| baseline_ovr_linearsvc            | 0.999500🟩      | 0.998800🟩      | 0.999200🟩             | 0.999800🟩          |
| baseline_ovr_calibrated_linearsvc | 0.999200        | 0.998400        | 0.998900               | 0.999500            |
| baseline_ovr_logreg               | 0.997300        | 0.995800        | 0.996000               | 0.998600            |
| baseline_ovr_sgd_logloss          | 0.995000🟥      | 0.992000🟥      | 0.993400🟥             | 0.996600🟥          |

IMPROVEMENT — VAL metrics
| model                | val_micro_f1   | val_macro_f1   |
|----------------------|----------------|----------------|
| ovr_logreg_bal_C2    | 0.980376🟩     | 0.965836🟩     |
| ovr_linearsvc_C1     | 0.980182       | 0.964417     

In [2]:
from pathlib import Path

def show_stage_str(title: str, rows: list[dict], mark_cols: list[str], sort_by: str | None = None) -> str:
    df = pd.DataFrame(rows)
    if df.empty:
        return f"\n{'='*90}\n{title}\n{'='*90}\n(Natija yo‘q)\n"

    if sort_by is not None and sort_by in df.columns:
        df = df.sort_values(sort_by, ascending=False).reset_index(drop=True)

    df_disp = mark_max_min(df, mark_cols)
    table = tabulate(df_disp, headers="keys", tablefmt="github", showindex=False)
    return f"\n{'='*90}\n{title}\n{'='*90}\n{table}\n"

all_out = []
all_out.append(show_stage_str("BASELINE — TEST metrics", baseline_rows,
                              ["test_micro_f1","test_macro_f1","test_micro_precision","test_micro_recall"], "test_micro_f1"))
all_out.append(show_stage_str("IMPROVEMENT — VAL metrics", improvement_rows,
                              ["val_micro_f1","val_macro_f1"], "val_micro_f1"))
all_out.append(show_stage_str("TUNING (NO_OVERSAMPLING) — VAL/TEST metrics", tuning_rows,
                              ["val_micro_f1","val_macro_f1","test_micro_f1","test_macro_f1"], "test_micro_f1"))
all_out.append(show_stage_str("BEST MODEL (FINAL TRAIN) — VAL/TEST metrics", best_model_rows,
                              ["val_micro_f1","val_macro_f1","test_micro_f1","test_macro_f1"], "test_micro_f1"))

# hard-coded best-of-stage ham bizda bor — shu 2 qatordan keyin qo‘shib yuboramiz:
df_best_of_stage = pd.DataFrame(best_of_stage_rows)
df_best_of_stage_disp = mark_max_min(df_best_of_stage, ["val_micro_f1","val_macro_f1","test_micro_f1","test_macro_f1"])
all_out.append("\n" + "="*90 + "\nBEST-OF-STAGE — yonma-yon taqqoslash (hard-coded)\n" + "="*90 + "\n" +
               tabulate(df_best_of_stage_disp, headers="keys", tablefmt="github", showindex=False) + "\n")

Path("compare_results_tabulate.md").write_text("".join(all_out), encoding="utf-8")
print("✅ Hammasi compare_results_tabulate.md fayliga yozildi (to‘liq ko‘rinadi).")

✅ Hammasi compare_results_tabulate.md fayliga yozildi (to‘liq ko‘rinadi).


In [ ]:
# 🟩 = shu bosqich ichida (shu metrikaning) eng yuqori qiymati
# 🟥 = shu bosqich ichida (shu metrikaning) eng past qiymati

# BASELINE (06_train_baseline.ipynb) — TEST metrics
# | model                             | test_micro_f1   | test_macro_f1   | test_micro_precision   | test_micro_recall   |
# |-----------------------------------|-----------------|-----------------|------------------------|---------------------|
# | baseline_ovr_linearsvc            | 0.9995🟩        | 0.9988🟩        | 0.9992🟩               | 0.9998🟩            |
# | baseline_ovr_calibrated_linearsvc | 0.9992          | 0.9984          | 0.9989                 | 0.9995              |
# | baseline_ovr_logreg               | 0.9973          | 0.9958          | 0.9960                 | 0.9986              |
# | baseline_ovr_sgd_logloss          | 0.9950🟥        | 0.9920🟥        | 0.9934🟥               | 0.9966🟥            |
# IMPROVEMENT (10_train_improvement.ipynb) — VAL metrics
# | model                | val_micro_f1   | val_macro_f1   |
# |----------------------|----------------|----------------|
# | ovr_logreg_bal_C2    | 0.980376🟩     | 0.965836🟩     |
# | ovr_linearsvc_C1     | 0.980182       | 0.964417       |
# | ovr_logreg_bal_C1    | 0.978403       | 0.944185       |
# | ovr_sgd_logloss      | 0.974033       | 0.952951       |
# | ovr_sgd_hinge        | 0.957535       | 0.936905       |
# | ovr_complementnb_a05 | 0.913360🟥     | 0.882115🟥     |
# TUNING (12_tuning_NO_OVERSAMPLING.ipynb) — VAL/TEST metrics
# | model              | best_score   | val_micro_f1   | val_macro_f1   | test_micro_f1   | test_macro_f1   |
# |--------------------|--------------|----------------|----------------|-----------------|-----------------|
# | optuna_logreg      | 0.962915     | 0.962941🟩     | 0.930969🟩     | 0.957416🟩      | 0.916767🟩      |
# | optuna_linearsvc   | 0.962932🟩   | 0.961265       | 0.908998       | 0.957016        | 0.912317        |
# | optuna_sgd_logloss | 0.951346     | 0.950963       | 0.916381       | 0.943415        | 0.903986        |
# | optuna_sgd_hinge   | 0.946947🟥   | 0.948021🟥     | 0.888889🟥     | 0.938087🟥      | 0.895398🟥      |
# BEST MODEL (FINAL TRAIN) — (12_tuning_NO_OVERSAMPLING.ipynb)
# | model                    | val_micro_f1   | val_macro_f1   | test_micro_f1   | test_macro_f1   |
# |--------------------------|----------------|----------------|-----------------|-----------------|
# | optuna_logreg_best_final | 0.984831🟩🟥   | 0.973785🟩🟥   | 0.978199🟩🟥    | 0.966022🟩🟥    |
# |------------------------------------------------------------------------------------------------------------
# (Bu bosqichda faqat 1 ta model bo‘lgani uchun u ham max, ham min bo‘lib chiqadi.)



# (Qo‘shimcha) Bosqichlar bo‘yicha “eng yaxshi”ni yonma-yon taqqoslash
# Bu jadvalda har bosqichning eng yaxshi modeli turibdi:

# | stage              | model                    | val_micro_f1   | val_macro_f1   | test_micro_f1   | test_macro_f1   |
# |--------------------|--------------------------|----------------|----------------|-----------------|-----------------|
# | baseline           | baseline_ovr_linearsvc  | —              | —              | 0.999500🟩      | 0.998800🟩      |
# | improvement        | ovr_logreg_bal_C2       | 0.980376       | 0.965836       | —               | —               |
# | tuning             | optuna_logreg           | 0.962941🟥     | 0.930969🟥     | 0.957416🟥      | 0.916767🟥      |
# | best_model (final) | optuna_logreg_best_final| 0.984831🟩     | 0.973785🟩     | 0.978199        | 0.966022        |
# Rang ma'nosi (shu jadval ichida):

# 🟩 = eng yuqori qiymat (shu ustunda)
# 🟥 = eng past qiymat (shu ustunda)
# — = bu bosqichda bu split mavjud emas (masalan baseline’da val yo‘q, improvement’da test yo‘q)    